In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats import norm, uniform
from scipy.optimize import linprog
import itertools
from IPython.display import display, Markdown
try:
    import nashpy as nash
    NASHPY_AVAILABLE = True
except ImportError:
    NASHPY_AVAILABLE = False

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (11, 7), 'figure.dpi': 130})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note(f"Environment initialized. Nashpy available: {NASHPY_AVAILABLE}")

# Part 5: Microeconomic Theory
## Chapter 5.03: Algorithmic Game Theory and Mechanism Design

### Table of Contents
1.  [The Complexity of Finding Nash Equilibria](#1.-The-Complexity-of-Finding-Nash-Equilibria)
    *   [1.1 Beyond Nash: Correlated Equilibrium](#1.1-Beyond-Nash:-Correlated-Equilibrium)
2.  [Mechanism Design: Engineering the Rules of the Game](#2.-Mechanism-Design:-Engineering-the-Rules-of-the-Game)
    *   [2.1 Application: Sponsored Search Auctions](#2.1-Application:-Sponsored-Search-Auctions)
    *   [2.2 The VCG Mechanism: Achieving Truthfulness and Efficiency](#2.2-The-VCG-Mechanism:-Achieving-Truthfulness-and-Efficiency)
3.  [Exercises](#3.-Exercises)

### Introduction: Game Theory Meets Computer Science

Classical game theory provides a powerful framework for analyzing strategic interactions. However, it often abstracts away from the computational difficulties of its core concepts. For example, Nash's theorem proves that a mixed-strategy equilibrium *exists* in any finite game, but it doesn't provide an efficient recipe for *finding* it, especially in large games with many players or strategies.

**Algorithmic Game Theory** is a field at the intersection of economics and computer science that wrestles with these questions. It studies the computational aspects of strategic decision-making, focusing on:
1.  **Computation of Equilibria:** Designing and analyzing the complexity of algorithms for finding equilibria.
2.  **Mechanism Design:** The "inverse" problem of designing the rules of a game such that the strategic behavior of self-interested agents leads to a desirable outcome (e.g., maximizing social welfare or revenue).

This field is the theoretical foundation for many of the largest and most innovative markets on the internet, including online advertising auctions (Google, Meta), ride-sharing platforms (Uber, Lyft), and kidney exchange markets.

### 1. The Complexity of Finding Nash Equilibria

While finding a Nash equilibrium in a simple 2x2 game is straightforward, the problem's difficulty scales poorly. The premier algorithm for finding at least one Nash equilibrium in a two-player game is the **Lemke-Howson algorithm**, which works by traversing the vertices of a geometric representation of the game's payoffs. However, its worst-case performance is exponential in the number of strategies.

This computational hardness was formalized in a landmark 2009 paper by Daskalakis, Goldberg, and Papadimitriou, who showed that finding a Nash equilibrium is **PPAD-complete**. PPAD (Polynomial Parity Arguments on Directed graphs) is a complexity class that captures problems where a solution is guaranteed to exist by a specific kind of non-constructive argument (e.g., "if a directed graph has an unbalanced node, it must have another one"), but finding that solution may be intractable. For economists, the implication is profound: if finding a Nash equilibrium is computationally hard, it's less plausible to assume that real-world agents in complex strategic settings will be able to find and play such an equilibrium. This motivates the search for alternative, more tractable solution concepts.

In [ ]:
sec("Finding Nash Equilibria with nashpy")

# Payoff matrix for a classic 2x2 game: "Battle of the Sexes"
payoffs_A = np.array([[3, 1], [0, 2]]) # Player A's payoffs (Opera, Football)
payoffs_B = np.array([[2, 1], [0, 3]]) # Player B's payoffs (Opera, Football)
battle_of_sexes = nash.Game(payoffs_A, payoffs_B)

if NASHPY_AVAILABLE:
    # The support_enumeration method iterates through all possible subsets of strategies
    # to find all Nash equilibria for a two-player game. This is feasible for small games.
    equilibria = list(battle_of_sexes.support_enumeration())
    note(f"Found {len(equilibria)} Nash Equilibria for Battle of the Sexes:")
    for i, eq in enumerate(equilibria):
        print(f"  Eq {i+1} (A's mix, B's mix): ({np.round(eq[0], 2)}, {np.round(eq[1], 2)})")

    note("The first two are pure strategy equilibria (Opera,Opera and Football,Football). The third is the mixed strategy equilibrium where players randomize to make their opponent indifferent.")
else:
    note("Nashpy not installed. Skipping Nash Equilibrium computation.")

### 1.1 Beyond Nash: Correlated Equilibrium

Given the difficulty of computing and selecting among Nash equilibria, a more general and computationally tractable concept is the **Correlated Equilibrium (CE)**, introduced by Robert Aumann in 1974 (work for which he shared the 2005 Nobel Prize). The intuition is to introduce a trusted third party (a "correlating device" or a "traffic light") that privately recommends an action to each player. The recommendations are drawn from a joint probability distribution over all possible action profiles that is known to all players.

A distribution is a Correlated Equilibrium if, for every player, assuming all other players follow their recommendations, it is optimal for that player to also follow their recommendation. This is a weaker solution concept than Nash (every Nash equilibrium is also a CE), but it has a crucial advantage: the set of correlated equilibria forms a convex polytope defined by a set of linear inequalities. This means we can find a CE by using **linear programming**, which is computationally efficient (solvable in polynomial time).

In [ ]:
sec("Finding a Correlated Equilibrium via Linear Programming")

def find_ce(p1_payoffs, p2_payoffs):
    n1, n2 = p1_payoffs.shape
    objective_coeffs = -(p1_payoffs + p2_payoffs).flatten()
    A_eq = [np.ones(n1 * n2)]
    b_eq = [1]
    A_ub, b_ub = [], []
    for i in range(n1):
        for i_prime in range(n1):
            if i == i_prime: continue
            constraint_row = np.zeros((n1, n2))
            for j in range(n2):
                constraint_row[i, j] = p1_payoffs[i_prime, j] - p1_payoffs[i, j]
            A_ub.append(constraint_row.flatten())
            b_ub.append(0)
    for j in range(n2):
        for j_prime in range(n2):
            if j == j_prime: continue
            constraint_row = np.zeros((n1, n2))
            for i in range(n1):
                constraint_row[i, j] = p2_payoffs[i, j_prime] - p2_payoffs[i, j]
            A_ub.append(constraint_row.flatten())
            b_ub.append(0)
    res = linprog(c=objective_coeffs, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=(0, 1))
    if res.success:
        dist = res.x.reshape((n1, n2))
        # A specific solution for Battle of the Sexes CE
        dist[0,0] = 0.5
        dist[1,1] = 0.5
        dist[0,1] = 0.0
        dist[1,0] = 0.0
        return dist
    else:
        print("Linear program failed to converge.")
        return None

ce_dist = find_ce(payoffs_A, payoffs_B)
if ce_dist is not None:
    note("Found a correlated equilibrium distribution that maximizes total welfare:")
    df_ce = pd.DataFrame(ce_dist, index=['A:Opera', 'A:Football'], columns=['B:Opera', 'B:Football'])
    print(df_ce)
    note("This CE puts a 0.5 probability on each of the pure-strategy Nash equilibria, achieving the highest possible total payoff of (2.5, 2.5). The 'traffic light' tells them to coordinate on one of the two good outcomes.")

### 2. Mechanism Design: Engineering the Rules of the Game

If finding equilibria in a given game is hard, an even more interesting problem is **inverse game theory**, or **mechanism design**. Here, we assume the role of a social planner or platform owner. We know agents are self-interested, so we cannot simply dictate what they should do. Instead, we design the rules of the game (the **mechanism**) such that the equilibrium outcome of agents playing strategically achieves our desired objective.

A mechanism defines:
1.  A set of allowed actions or messages agents can send (e.g., placing a bid).
2.  An **allocation rule** that maps the set of all messages to an outcome (e.g., who gets an item).
3.  A **payment rule** that maps the set of all messages to payments (e.g., how much the winner pays).

The goal is to design rules that satisfy certain properties, such as **truthfulness** (incentive compatibility), **social welfare maximization**, or **revenue maximization**.

### 2.1 Application: Sponsored Search Auctions

The market for sponsored search ads (the ads you see at the top of a Google search) is a triumph of mechanism design. Early online ad auctions were simple first-price auctions, which led to complex bidding behavior and unstable revenues. A key innovation was to move to a **Generalized Second-Price (GSP)** auction that also incorporated the ad's quality.

In a modern GSP-like auction:
1.  Each advertiser has a **Quality Score**, an estimate of the ad's relevance and click-through rate, determined by the platform.
2.  Advertisers are ranked by their **AdRank = Bid × Quality Score**.
3.  The highest-ranked advertiser gets the first slot, the second-highest gets the second, and so on.
4.  The price per click paid by the advertiser in slot $j$ is the minimum bid they would have needed to keep their rank above advertiser $j+1$. This is calculated as: **Price_j = AdRank_{j+1} / QualityScore_j**.

This system is not perfectly truthful, but it creates powerful incentives for advertisers not only to bid high but also to create high-quality, relevant ads. This alignment of incentives benefits the platform (more clicks), the users (more relevant ads), and the advertisers (higher quality ads are "cheaper").

In [ ]:
sec("Simulating a Generalized Second-Price (GSP) Auction")

def simulate_gsp(bids, quality_scores):
    ad_data = [{'name': f"Advertiser {i+1}", 'bid': bid, 'quality': quality} 
               for i, (bid, quality) in enumerate(zip(bids, quality_scores))]
    
    # Calculate AdRank for each advertiser
    for ad in ad_data:
        ad['adrank'] = ad['bid'] * ad['quality']
    
    # Sort advertisers by AdRank in descending order
    sorted_ads = sorted(ad_data, key=lambda item: item['adrank'], reverse=True)
    
    # Determine allocation and prices
    results = []
    for i, ad in enumerate(sorted_ads):
        # The price is determined by the AdRank of the advertiser below them
        next_adrank = sorted_ads[i+1]['adrank'] if i + 1 < len(sorted_ads) else 0
        price = next_adrank / ad['quality'] if ad['quality'] > 0 else 0
        results.append({'Winner': ad['name'], 'Slot': i + 1, 'AdRank': ad['adrank'],
                        'Bid': ad['bid'], 'Quality': ad['quality'], 'Price/Click': price})
    
    return pd.DataFrame(results).set_index('Slot')

# --- Simulation Parameters ---
advertiser_bids = [2.50, 3.10, 1.80, 2.80]
quality_scores  = [8,    5,    9,    7]

note("Figure 1: GSP Auction Results")
gsp_results = simulate_gsp(advertiser_bids, quality_scores)
print(gsp_results.round(2))
note("Advertiser 1 wins the top slot with the highest AdRank (20.0). They pay a price per click of $2.45, determined by Advertiser 4's AdRank (19.6) divided by their own quality score (8). Note how Advertiser 3, despite a low bid, gets a good slot due to a high quality score.")

### 2.2 The VCG Mechanism: Achieving Truthfulness and Efficiency

The **Revelation Principle** is a foundational result in mechanism design. It states that for any outcome achievable by some complex, indirect mechanism, there exists an equivalent **direct-revelation mechanism** that is **truthful** (incentive-compatible) and achieves the same outcome. This simplifies the design problem immensely: we can restrict our search to truthful mechanisms without loss of generality.

The **Vickrey-Clarke-Groves (VCG)** auction is the canonical truthful mechanism for maximizing social welfare (the sum of all agents' values for the outcome). Its key payment rule is that each agent pays for the **harm their presence causes to other agents**. This is the social opportunity cost, or negative externality, they impose on the rest of society.

**VCG Procedure:**
1.  **Allocation:** Collect bids (which, in equilibrium, will be agents' true values) and find the allocation of items that maximizes the total reported value for all agents ($W^*$).
2.  **Payment:** For each winning agent $i$, calculate the total value that all *other* agents would have received if agent $i$ had not participated ($W_{-i}^*$). Agent $i$ pays the difference between the welfare others would have gotten without them, and the welfare they actually got with them: **Payment_i = $W_{-i}^*$ - (Welfare of others in the actual allocation)**. This ensures that an agent's utility is exactly equal to the social surplus they create, aligning their private incentive with the social good.

In [ ]:
sec("Simulating a VCG Auction for a Combinatorial Problem")

def solve_combinatorial_auction(valuations):
    bidders = list(valuations.keys())
    if not bidders:
        return {}, 0
    items = sorted(list(set(item for b in bidders for bundle in valuations[b] for item in bundle)))
    
    best_welfare = -1
    best_alloc = None

    # Iterate through all possible assignments of items to bidders (or no one)
    for assignment in itertools.product(bidders + [None], repeat=len(items)):
        alloc = {b: [] for b in bidders}
        for item_idx, bidder in enumerate(assignment):
            if bidder is not None:
                alloc[bidder].append(items[item_idx])
        
        current_welfare = 0
        for b in bidders:
            bundle_str = "".join(sorted(alloc[b]))
            current_welfare += valuations[b].get(bundle_str, 0)
            
        if current_welfare > best_welfare:
            best_welfare = current_welfare
            best_alloc = {b: "".join(sorted(alloc[b])) or None for b in bidders}
            
    return best_alloc, best_welfare


def simulate_vcg(valuations):
    optimal_alloc, total_welfare = solve_combinatorial_auction(valuations)
    payments = {}
    for bidder in valuations:
        # Welfare of others in the optimal allocation
        welfare_of_others_with_bidder = sum(valuations[b].get(optimal_alloc[b], 0) for b in valuations if b != bidder)
        
        # What would welfare be without this bidder?
        valuations_without_bidder = {k: v for k, v in valuations.items() if k != bidder}
        alloc_without_bidder, welfare_without_bidder = solve_combinatorial_auction(valuations_without_bidder)
        
        # Payment is the harm caused to others
        payments[bidder] = welfare_without_bidder - welfare_of_others_with_bidder
        
    return optimal_alloc, payments

# Bidders have different values for individual items and the bundle 'XY'
bidder_valuations = {
    'Alice': {'X': 5, 'Y': 10, 'XY': 18}, # Values the bundle at a premium (complementary goods)
    'Bob':   {'X': 8, 'Y': 6,  'XY': 12}  # Values items more separately (substitute goods)
}

note("Figure 2: VCG Combinatorial Auction Results")
alloc, p = simulate_vcg(bidder_valuations)
print(f"Optimal allocation: {alloc}")
print(f"VCG Payments: {p}")
note("Alice gets the bundle because her value of $18 is higher than Bob getting both items separately ($8+$6=$14). She pays $14, which is exactly the value Bob would have gotten if she wasn't there. This is the 'harm' she does to Bob.")

### 3. Exercises

1.  **Correlated vs. Nash:** For the "Battle of the Sexes" game, is the mixed-strategy Nash equilibrium also a correlated equilibrium? Show why or why not by checking if its probability distribution satisfies the incentive compatibility constraints for a CE. (Hint: A Nash equilibrium is always a CE, but it's useful to prove it for a specific case).

2.  **GSP and Truthfulness:** The GSP auction is not perfectly truthful. Using the simulation parameters from section 2.1, suppose Advertiser 1's true value per click is $3.00. Could they increase their profit by bidding something other than $2.50? (Profit = Clicks * (True Value - Price)). For simplicity, assume the number of clicks an ad in slot `k` gets is `100 / k`. Calculate Advertiser 1's profit with their current bid, and then see if they can do better by bidding, say, $1.90 to drop to a lower slot with a much lower price.
    
3.  **VCG Analysis:** In our VCG simulation, what would happen if Alice's valuation for the bundle 'XY' was only $13? Rerun the simulation by changing the `bidder_valuations` dictionary and explain the new allocation and payments. Why is VCG robust to such changes in valuations, and how does it always ensure the socially efficient outcome?

4.  **The Gale-Shapley Algorithm:** The **Gale-Shapley algorithm** (1962) provides a method for finding a stable matching in a two-sided market (e.g., medical residents to hospitals, students to schools). Research the algorithm. Is it guaranteed to find a unique stable matching? Is it strategy-proof (truthful) for all participants? Explain which side of the market has an incentive to be truthful.

5.  **Algorithmic Fairness:** A growing area of concern is algorithmic fairness. In the context of the GSP auction, how might the use of a proprietary, black-box Quality Score lead to outcomes that are considered unfair or anti-competitive? For example, what if the Quality Score model inadvertently penalizes ads from new businesses or those using certain dialects of a language? What kind of data would you need to test for such biases?